In [6]:
''' generating ground truth speech
Use speechbrain cond env
'''
import os
import random
import string
import pandas as pd
import glob
from collections import defaultdict
from tqdm import tqdm
from IPython.display import Audio

import torch
import torchaudio
import soundfile as sf
from datasets import load_dataset

from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5ForSpeechToSpeech, SpeechT5HifiGan 
from speechbrain.pretrained import EncoderClassifier
from datasets import load_dataset
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
def simple_clean_text(text):
    # lower case, remove punctuation
    return text.translate(str.maketrans('', '', string.punctuation)).lower().strip()

def get_gt():
    # get ground truth from csv file
    my_dict = defaultdict(dict)
    df = pd.read_excel('Harvard_Sentences_Keywords_CASALab_homophones.xlsx')
    for idx, row in df.iterrows():
        list_id = int(row['LIST'])
        text = simple_clean_text(row['SENTENCE']).split()
        sent_id, *sent = simple_clean_text(row['SENTENCE']).split()
        my_dict[list_id][int(sent_id)] = ' '.join(sent)
    return my_dict

In [8]:
def get_tts_models():
    processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
    model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
    vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
    embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
    classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb", run_opts={"device":device})
    return processor, model.to(device), vocoder.to(device), embeddings_dataset, classifier

def get_vc_models():
    processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_vc")
    model = SpeechT5ForSpeechToSpeech.from_pretrained("microsoft/speecht5_vc")
    vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")
    classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb", run_opts={"device":device})
    return processor, model.to(device), vocoder.to(device), classifier
    
def tts():
    processor, model, vocoder, embeddings_dataset, classifier = get_tts_models()
    gt_dict = get_gt()
    for list_id, list_dict in tqdm(gt_dict.items()):
        for sent_id, sent in list_dict.items():
            try:
                file_name = f'{list_id}_{sent_id}'
                inputs = processor(text=sent, return_tensors="pt")
                inputs = {key: val.to(device) for key, val in inputs.items()}
                # load xvector containing speaker's voice characteristics from a dataset
                rand_speaker_id = random.randint(0, len(embeddings_dataset)-1)
                speaker_embeddings = torch.tensor(embeddings_dataset[rand_speaker_id]["xvector"]).unsqueeze(0)
                speech = model.generate_speech(inputs["input_ids"], speaker_embeddings.to(device), vocoder=vocoder)
                sf.write(f"data/ground_truth/{file_name}.wav", speech.cpu().numpy(), samplerate=16000)
            except Exception as e:
                print(e)
                print(f'Cannot TTS for {file_name}: {sent}')

def tts_speaker_voice():
    filenames = glob.glob('data/no-noise_16k/*.wav')
    save_dir = 'data/ground_truth_speaker_voice_16k'
    if not os.path.exists(save_dir): os.mkdir(save_dir)
    processor, model, vocoder, embeddings_dataset, classifier = get_tts_models()
    gt_dict = get_gt()
    for filename in tqdm(filenames):
        basename = os.path.basename(filename).replace('.wav', '')
        savename = os.path.join(save_dir, f'{basename}.wav')
        try:
            # GET SENTENCE
            if "70db" in basename: # "sim_oc11_habitual_nm_h15_10_ch1_70db"
                list_id = basename.split("_")[4][1:]  # h15 -> 15
                sent_id = basename.split("_")[5]
            else: # "sim_NM_amp_babble_pd02_h17_1"
                list_id = basename.split("_")[5][1:]  # h17 -> 17
                sent_id = basename.split("_")[6]
            list_id, sent_id = int(list_id), int(sent_id)
            sent = gt_dict[list_id][sent_id]
            # GET VOICE EMBEDDINGS
            signal, fs =torchaudio.load(filename)
            assert fs == 16000, fs
            speaker_embeddings = classifier.encode_batch(signal).squeeze(0)
            # TTS
            inputs = processor(text=sent, return_tensors="pt")
            inputs = {key: val.to(device) for key, val in inputs.items()}
            speech = model.generate_speech(inputs["input_ids"], speaker_embeddings.to(device), vocoder=vocoder)
            sf.write(savename, speech.cpu().numpy(), samplerate=16000)
        except Exception as e:
            print(e)
            print(f'Cannot TTS for {file_name}: {sent}')

def vc_speaker_voice():
    # https://huggingface.co/microsoft/speecht5_vc
    filenames = glob.glob('data/no-noise_16k/*.wav')
    save_dir = 'data/ground_truth_speaker_voice_16k_v2'
    if not os.path.exists(save_dir): os.mkdir(save_dir)
    processor, model, vocoder, classifier = get_vc_models()
    for voicename in tqdm(filenames):
        basename = os.path.basename(voicename).replace('.wav', '')
        savename = os.path.join(save_dir, f'{basename}.wav')
        try:
            # GET IDs
            if "70db" in basename: # "sim_oc11_habitual_nm_h15_10_ch1_70db"
                list_id = basename.split("_")[4][1:]  # h15 -> 15
                sent_id = basename.split("_")[5]
            else: # "sim_NM_amp_babble_pd02_h17_1"
                list_id = basename.split("_")[5][1:]  # h17 -> 17
                sent_id = basename.split("_")[6]
            list_id, sent_id = int(list_id), int(sent_id)
            # GET FILE SIGNAL
            filename = f'data/ground_truth/{list_id}_{sent_id}.wav'
            signal, fs =torchaudio.load(filename)
            assert fs == 16000, fs
            inputs = processor(audio=signal, sampling_rate=fs, return_tensors="pt")
            inputs = {key: val.to(device) for key, val in inputs.items()}
            # GET VOICE EMBEDDINGS
            signal_voice, fs_voice =torchaudio.load(voicename)
            assert fs_voice == 16000, fs_voice
            speaker_embeddings = classifier.encode_batch(signal_voice).squeeze(0)
            # VOICE CONVERSION
            speech = model.generate_speech(inputs["input_values"].squeeze(0), speaker_embeddings, vocoder=vocoder)
            sf.write(savename, speech.cpu().numpy(), samplerate=16000)
        except Exception as e:
            print(e)
            print(f'Cannot TTS for {file_name}: {sent}')

In [18]:
processor, model, vocoder, embeddings_dataset, classifier = get_tts_models()

Found cached dataset cmu-arctic-xvectors (/home/hanqing/.cache/huggingface/datasets/Matthijs___cmu-arctic-xvectors/default/0.0.1/a62fea1f9415e240301ea0042ffad2a3aadf4d1caa7f9a8d9512d631723e781f)


In [27]:
with open('./loco_4_extracted_wavs/groundtruth_transcript.txt', 'r') as f:
    lines = f.readlines()
    gt = [l.strip().split(',') for l in lines]
files = os.listdir('loco_4_extracted_wavs/noisy_speech_16k')
files = set([f.replace('.wav', '') for f in files])
gt_dict = {k: simple_clean_text(v) for (k, v) in gt if k in files}
assert len(gt_dict)==len(files)
des_dir = './loco_4_extracted_wavs/gt_speech_16k'
for f_id, sent in tqdm(gt_dict.items()):
    try:
        inputs = processor(text=sent, return_tensors="pt")
        inputs = {key: val.to(device) for key, val in inputs.items()}
        # load xvector containing speaker's voice characteristics from a dataset
        rand_speaker_id = random.randint(0, len(embeddings_dataset)-1)
        speaker_embeddings = torch.tensor(embeddings_dataset[rand_speaker_id]["xvector"]).unsqueeze(0)
        speech = model.generate_speech(inputs["input_ids"], speaker_embeddings.to(device), vocoder=vocoder)
        sf.write(f"{des_dir}/{f_id}.wav", speech.cpu().numpy(), samplerate=16000)
    except Exception as e:
        print(e)
        print(f'Cannot TTS for {f_id}: {sent}')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [04:02<00:00,  1.57it/s]


In [30]:
processor, model, vocoder, classifier = get_vc_models()

In [41]:
# https://huggingface.co/microsoft/speecht5_vc
filenames = glob.glob('loco_4_extracted_wavs/noisy_speech_16k/*.wav')
save_dir = 'loco_4_extracted_wavs/gt_speech_16k_vocal'
# if not os.path.exists(save_dir): os.mkdir(save_dir)
for voicename in tqdm(filenames):
    basename = os.path.basename(voicename).replace('.wav', '')
    savename = os.path.join(save_dir, f'{basename}.wav')
    try:
#         print(basename, '\n\t', savename)
        # GET FILE SIGNAL
        filename = f'loco_4_extracted_wavs/gt_speech_16k/{basename}.wav'
        assert os.path.exists(filename)
        signal, fs =torchaudio.load(filename)
        assert fs == 16000, fs
        inputs = processor(audio=signal, sampling_rate=fs, return_tensors="pt")
        inputs = {key: val.to(device) for key, val in inputs.items()}
        # GET VOICE EMBEDDINGS
        signal_voice, fs_voice =torchaudio.load(voicename)
        assert fs_voice == 16000, fs_voice
        speaker_embeddings = classifier.encode_batch(signal_voice).squeeze(0)
        # VOICE CONVERSION
        speech = model.generate_speech(inputs["input_values"].squeeze(0), speaker_embeddings, vocoder=vocoder)
        sf.write(savename, speech.cpu().numpy(), samplerate=16000)
    except Exception as e:
        print(e)
        print(f'Cannot TTS for {voicename}: {sent}')

  0%|                                                                                                                                                                               | 0/380 [00:00<?, ?it/s]

torch.Size([1, 1, 512])


  0%|▍                                                                                                                                                                      | 1/380 [00:00<02:50,  2.22it/s]

torch.Size([1, 1, 512])


  1%|▉                                                                                                                                                                      | 2/380 [00:01<04:13,  1.49it/s]

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 378/380 [03:47<00:01,  1.90it/s]

torch.Size([1, 1, 512])


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 379/380 [03:47<00:00,  2.11it/s]

torch.Size([1, 1, 512])


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 380/380 [03:48<00:00,  1.66it/s]


In [ ]:
# tts();

In [ ]:
# tts_speaker_voice();

In [4]:
vc_speaker_voice();

  0%|                                                                                                                                                                             | 0/405 [00:00<?, ?it/s]

torch.Size([1, 1, 512])


  0%|▍                                                                                                                                                                    | 1/405 [00:01<09:00,  1.34s/it]

torch.Size([1, 1, 512])


  0%|▊                                                                                                                                                                    | 2/405 [00:01<05:54,  1.14it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 403/405 [03:22<00:00,  2.19it/s]

torch.Size([1, 1, 512])


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 404/405 [03:23<00:00,  2.16it/s]

torch.Size([1, 1, 512])


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 405/405 [03:23<00:00,  1.99it/s]


In [ ]:
# draft

In [7]:
name = 'data/ground_truth_speaker_voice_16k_v2/sim_NM_amp_nn_pd04_h15_10.wav'
base = os.path.basename(name)
dirname = os.path.dirname(name)
Audio(name)

In [8]:
Audio(f'data/no-noise_16k/{base}')